In [2]:
import numpy as np
import matplotlib.gridspec
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, IntSlider, HBox, Layout, Output, VBox
import ipywidgets as widgets
from scipy import signal

%matplotlib widget

In [4]:
def unit_circle():
    x1 = np.linspace(-1, 1, 1000)
    y1 = np.sqrt(1-x1**2)
    x2 = np.linspace(0.999, -1, 1000)
    y2 = np.sqrt(1-x2**2)
    x = np.concatenate([x1, x2])
    y = np.concatenate([y1, -y2])
    labels = ['-2j', '-1.5j', '-j', '-0.5j', '0', '0.5j', 'j','1.5j','2j']
    position=[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
    plt.yticks(position, labels, rotation='horizontal')
    return x, y

In [5]:
plt.figure(figsize=(5,5))
uc = unit_circle()

# Pad margins so that markers don't get clipped by the axes
#plt.margins(5)
# Tweak spacing to prevent clipping of tick-labels
#plt.subplots_adjust(bottom=0.5)
plt.plot(uc[0], uc[1])
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
x = []
y = []

plt.close('all')
fig, ax = plt.subplots(figsize=(5,5))
uc = unit_circle()
ax.plot(uc[0], uc[1])
ax.grid()
ax.plot(x,y, 'xr')
x = [0.5, 0.5, 0.5, 0.5, 0.5]
y = -np.array([0.5, 0.5, 0.5, 0.5, 0.5])
ax.plot(x, y, 'ob', fillstyle='none')
ax.set_xlim([-2,2])
ax.set_ylim([-2,2])

class DraggableMarker():
    def __init__(self, ax=None, lines=None, update_func=None):
        if ax == None:
            self.ax = plt.gca()
        else:
            self.ax=ax
        if lines==None:
            self.lines=self.ax.lines
        else:
            self.lines=lines
        self.lines = self.lines[:]
        for line in self.lines:
            x, y = line.get_data()
            line.set_data(x.astype(np.float64), y.astype(np.float64))
        
        self.update_func = update_func
        self.tx =  self.ax.text(0,0,"")
        
        self.active_point = 0
        self.active_line = 0

        self.draggable=False
        
        self.c1 = self.ax.figure.canvas.mpl_connect("button_press_event", self.click)
        self.c2 = self.ax.figure.canvas.mpl_connect("button_release_event", self.release)
        self.c3 = self.ax.figure.canvas.mpl_connect("motion_notify_event", self.drag)
    
    def get_active_line(self):
        return self.active_line
    
    def get_active_point(self):
        return self.active_point
    
    def click(self,event):
        if event.button==1:
            #leftclick
            self.draggable=True
        elif event.button==3:
            self.draggable=False
            self.tx.set_visible(False)
            self.ax.figure.canvas.draw_idle() 
            return
        
        self.active_point, self.active_line = self.get_closest(event.xdata, event.ydata)  
        if self.active_point is None or self.active_line is None:
            self.draggable = False
            return
        
        self.tx.set_visible(True)
        self.update(event)
        self.ax.figure.canvas.draw_idle() 
        
    def drag(self, event):
        if self.draggable:
            self.update(event)
            self.ax.figure.canvas.draw_idle()

    def release(self,event):
        self.draggable=False
        
    def update(self, event):
        self.tx.set_position((event.xdata, event.ydata))
        self.tx.set_text(f"Re: {event.xdata:.3f}\nIm: {event.ydata:.3f}")
        data_x, data_y = self.lines[self.active_line].get_data()
        data_x[self.active_point] = event.xdata
        data_y[self.active_point] = event.ydata
        self.lines[self.active_line].set_data(data_x, data_y)
        if self.update_func is not None:
            self.update_func()
        
        # Update transfer function
            
    def get_closest(self, mx, my):
        min_dist = np.iinfo(np.int64).max
        line_idx = None
        min_idx = None
        for i, line in enumerate(self.lines):
            x, y = line.get_data()
            # Check for empty lines
            if x.size == 0 or y.size == 0:
                continue
            dist = (x-mx)**2+(y-my)**2
            new_min_dist = np.min(dist)
            if new_min_dist < min_dist and new_min_dist < 0.0625:
                min_dist = new_min_dist
                min_idx = np.argmin(dist)
                line_idx = i
        
        return min_idx, line_idx
    
dm = DraggableMarker(lines=ax.lines[1:])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
dm.lines[1].get_data()

In [ ]:
a = np.array([])
if a.size == 0:
    print('Empty')

In [ ]:
poles = np.array([3-1j, 4+1j])
zeros = np.array([0-0.2j, 2-0.5j])
gain = np.prod(1-poles) / np.prod(1-zeros)
H = signal.ZerosPolesGain(zeros, poles, gain, dt=0.1).to_tf()

# Continuous
w, mag, phase = H.bode()
plt.close('all')
plt.figure()
plt.subplot(211)
plt.title('Bode plot for continuous case')
plt.plot(w, mag)    # Bode magnitude plot
plt.subplot(212)
plt.plot(w, phase)  # Bode phase plot
plt.show()

# Discrete
w, h = signal.freqz(H.num, H.den)
w = np.concatenate([-w[::-1], w])
h = np.concatenate([h[::-1], h])
plt.figure()
plt.title('Frequency response for discrete case')
plt.plot(w, 20*abs(h))
plt.xlabel('$\omega$ [rad]')
plt.ylabel('Gain [dB]')
plt.show()

In [72]:
class DraggableZeroPolePlot(DraggableMarker):
    def __init__(self, zeros=1, poles=1, show_phase=None, continuous=False):
        self.out = Output(layout={'width': '980px', 'height': '450px'})
        self.axs=[]
        self.discrete_mode = not continuous
        self.show_phase = not self.discrete_mode if show_phase is None else show_phase
        self.actual_change = True
        # Initialize zeros and poles
        z_x = []
        z_y = []
        for i in range(zeros):
            z_x.append(0.5)
            z_y.append(0.5)
        p_x = []
        p_y = []
        for i in range(poles):
            p_x.append(0.5)
            p_y.append(-0.5)
        # Initialize the figure with the initial number of zeros and poles
        self.init_figure(z_x, z_y, p_x, p_y)
        # Call the super class with the zero pole axis to enable the draggable markers
        super().__init__(ax=self.axs[0], lines=self.axs[0].lines[1:], update_func=self.change_freq_res)
        
        # Debug text field
        self.tx_deb = self.axs[0].text(-2,0,'')
        
        # 'Calculation not possible' text fields
        self.cnp_gain = None
        self.cnp_ph = None
        
        # Text field numbers
        self.lastzeroRe = 0
        self.lastzeroIm = 0
        
        # Init frequency response plot      
        self.change_freq_res(init=True)

        # Widgets
        # Zeros
        self.zero_range = widgets.IntSlider(value=zeros, min=0, max=zeros+10, step=1, description='Zeros:')
        
        self.zero_range.observe(self.on_zero_change, names='value')
        # Poles
        self.pole_range = widgets.IntSlider(value=poles, min=0, max=poles+10, step=1, description='Poles:')

        self.pole_range.observe(self.on_pole_change, names='value')
        
        # Check box to show phase plot
        self.phase_check = widgets.Checkbox(value=self.show_phase, description='Show phase plot')
        self.phase_check.observe(self.show_phase_callback, names='value')
        
        # Button to switch between continuous and discrete mode
        self.mode_button = widgets.Button(description= 'Switch to continuous' if self.discrete_mode else 'Switch to discrete',
                                         layout=Layout(width='20%'))
        self.mode_button.on_click(self.mode_button_callback)
        
        #Float text widgets
        self.input_Zero_RE=widgets.FloatText(value=self.lastzeroRe, description='Re part:')
        self.input_Zero_RE.observe(self.Zero_RE_Caller, names='value')
        
        self.input_Zero_IM=widgets.FloatText(value=self.lastzeroIm, description='Im part:')
        self.input_Zero_IM.observe(self.Zero_IM_Caller, names='value')
        
        # Display widgets and plot
        display(VBox([self.out, 
                      HBox([self.zero_range, self.pole_range, self.mode_button, self.phase_check]), 
                      HBox([self.input_Zero_RE, self.input_Zero_IM])]))
        plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    def init_figure(self, z_x, z_y, p_x, p_y):
        with self.out:
            # Create the zero pole plot
            self.fig = plt.figure(figsize=(8, 4))
            self.gs = self.fig.add_gridspec(2, 2)
            self.axs.append(self.fig.add_subplot(self.gs[:,0]))
            uc = self.unit_circle()
            # Draw unit circle
            self.axs[0].plot(uc[0], uc[1], color='black', linewidth='0.5')
            if not self.discrete_mode:
                self.axs[0].lines[0].set_visible(False)
            # Add zeros and poles
            self.axs[0].plot(z_x, z_y, 'xr', label='Zeros')
            self.axs[0].plot(p_x, p_y, 'ob', fillstyle='none', label='Poles')
            self.axs[0].set_xlim([-2, 2])
            self.axs[0].set_ylim([-2, 2])
            # Display the real and imaginary axes
            self.axs[0].set_yticks([1e-4], minor=True)
            self.axs[0].yaxis.grid(True, which='minor')
            self.axs[0].set_xticks([1e-4], minor=True)
            self.axs[0].xaxis.grid(True, which='minor')
            self.axs[0].set_title('Discrete Zero Pole Plot' if self.discrete_mode else 'Continuous Zero Pole Plot')
            self.axs[0].set_xlabel('Re')
            self.axs[0].set_ylabel('Im')
            # Enable the legend
            self.axs[0].legend()  
    
    def mode_button_callback(self, value):
        self.discrete_mode = self.mode_button.description == 'Switch to discrete'
        self.show_phase = not self.discrete_mode
        self.phase_check.value = self.show_phase
        self.mode_button.description = 'Switch to continuous' if self.discrete_mode else 'Switch to discrete'
        self.change_freq_res(init=True, redraw=True)
    
    def show_phase_callback(self, value):
        self.show_phase = value['new']
        self.change_freq_res(init=True, redraw=True)
    
    def Zero_RE_Caller(self, change):
        if self.actual_change:
            x_min, x_max = self.axs[0].get_xlim()
            self.lastzeroRe = np.clip(change['new'], x_min, x_max)
            self.ChangeZero()
    
    def Zero_IM_Caller(self, change):
        if self.actual_change:
            y_min, y_max = self.axs[0].get_ylim()
            self.lastzeroIm = np.clip(change['new'], y_min, y_max)
            self.ChangeZero()
        
    def ChangeZero(self):
        l_x, l_y = self.axs[0].lines[self.active_line+1].get_data()
        l_x[self.active_point]=self.lastzeroRe
        l_y[self.active_point]=self.lastzeroIm
        self.axs[0].lines[self.active_line+1].set_data(l_x, l_y)
        self.tx.set_position((self.lastzeroRe, self.lastzeroIm))
        self.tx.set_text(f"Re: {self.lastzeroRe:.3f}\nIm: {self.lastzeroIm:.3f}")
        self.change_freq_res()
    
    def on_zero_change(self, change):
        if change['new'] < change['old']:
            x, y = self.axs[0].lines[1].get_data()
            x = x[:-1]
            y = y[:-1]
            self.axs[0].lines[1].set_data(x, y)
        else:
            x, y = self.axs[0].lines[1].get_data()
            x = np.append(x, 0.5)
            y = np.append(y, 0.5)
            self.axs[0].lines[1].set_data(x, y)
        # Update frequency response plot
        self.change_freq_res()
    
    def on_pole_change(self, change):
        if change['new'] < change['old']:
            x, y = self.axs[0].lines[2].get_data()
            x = x[:-1]
            y = y[:-1]
            self.axs[0].lines[2].set_data(x, y)
        else:
            x, y = self.axs[0].lines[2].get_data()
            x = np.append(x, 0.5)
            y = np.append(y, -0.5)
            self.axs[0].lines[2].set_data(x, y)
        # Update frequency response plot
        self.change_freq_res()
                
    def change_freq_res(self, init=False, redraw=False):
        if self.cnp_gain is not None:
            self.cnp_gain.remove()
            self.cnp_gain = None
        if self.cnp_ph is not None:
            self.cnp_ph.remove()
            self.cnp_ph = None
        
        if init == True:
            # Generate the plots from scratch and name the axes
            with self.out:
                if redraw == True:
                    # Remove the gain and phase plots
                    for i in range(1, len(self.axs)):
                        self.axs[1].remove()
                        self.axs.pop(1)
                self.axs[0].lines[0].set_visible(self.discrete_mode)
                # Add gain (and phase) plot
                if self.show_phase:
                    self.axs.append(self.fig.add_subplot(self.gs[:1,1]))
                    self.axs.append(self.fig.add_subplot(self.gs[1:,1]))
                else:
                    self.axs.append(self.fig.add_subplot(self.gs[:,1]))
        
        # Get zeros and poles from the zero pole plot
        z_re, z_im = self.axs[0].lines[1].get_data()
        z = z_re + 1j*z_im
        p_re, p_im = self.axs[0].lines[2].get_data()
        p = p_re + 1j*p_im
        # Calculate the gain (C)
        gain = np.prod(1-z) / np.prod(1-p)            
        
        if self.discrete_mode:
            # Generate the transfer function
            H = signal.ZerosPolesGain(z, p, gain, dt=0.1).to_tf()
            # Generate dicrete frequency response
            w, h = signal.freqz(H.num, H.den, whole=True)
            #w,h=signal.freqresp(H)
            # Shift the angles to [-pi, pi]
            w = w-np.pi
            # Shift the gain and phase accordingly
            h = 20*np.log10(abs(np.fft.fftshift(h)))
            h_ph = np.fft.fftshift(np.angle(h, deg=True))
            #h=np.abs(h)
            #h_ph=np.angle(H)
        else:
            # Generate the transfer function
            H = signal.ZerosPolesGain(z, p, gain).to_tf()
            # Generate the continuous frequency response
            
            try:
                #w, h, h_ph = H.bode(n=1000)
                w1,h11 = H.freqresp(n=1000)
                h1=np.abs(h11)
                h_ph1=np.angle(h11)
                w2=np.flip(-w1)
                w2,h21 = H.freqresp(w2,n=1000)
                #h21=np.flip(w1)
                h2=np.abs(h21)
                h_ph2=np.angle(h21)
                
                
                w=np.concatenate((w2,[0.],w1))
                h=np.concatenate((h2,[0.],h1))
                h_ph=np.concatenate((h_ph2,[0.],h_ph1))
                
            except ValueError:
                w = 1
                h = 1
                h_ph = 1
                self.cnp_gain = self.axs[1].text(0.5, 0.5, 'Calculation not possible', fontdict={'color':'red', 'size':17}, 
                                                 horizontalalignment='center',
                                                 verticalalignment='center', 
                                                 transform=self.axs[1].transAxes)
                if self.show_phase:
                    self.cnp_ph = self.axs[2].text(0.5, 0.5, 'Calculation not possible', fontdict={'color':'red', 'size':17}, 
                                                   horizontalalignment='center',
                                                   verticalalignment='center', 
                                                   transform=self.axs[2].transAxes)

        if init == True:
            with self.out:
                # Gain
                self.axs[1].set_title('Frequency response')
                if self.discrete_mode:
                    self.axs[1].plot(w, h)
                else:
                    self.axs[1].plot(w, h)
                self.axs[1].set_xlabel('$\omega$ [rad]')
                self.axs[1].set_ylabel('Gain [dB]')
                # Phase
                if self.show_phase:
                    if self.discrete_mode:
                        self.axs[2].plot(w, h_ph)
                    else:
                        self.axs[2].plot(w, h_ph)
                    self.axs[2].set_xlabel('$\omega$ [rad]')
                    self.axs[2].set_ylabel('phase [deg]')
        else:
            # Only change the values of the plots
            # Gain
            self.axs[1].lines[0].set_data(w, h)  
            h_min = np.min(h)
            h_max = np.max(h)
            h_range = abs(h_max - h_min)
            self.axs[1].set_ylim([h_min-0.05*h_range, h_max+0.05*h_range] if h_min != h_max else [h_min-1, h_max+1])
            w_min = np.min(w)
            w_max = np.max(w)
            w_range = abs(w_max - w_min)
            self.axs[1].set_xlim([w_min-0.05*w_range, w_max+0.05*w_range] if w_min != w_max and w_min-0.05*w_range > 0
                                 else [w_min, w_max+1])
            # Phase
            if self.show_phase:
                h_ph_min = np.min(h_ph)
                h_ph_max = np.max(h_ph)
                h_ph_range = abs(h_ph_max - h_ph_min)
                self.axs[2].lines[0].set_data(w, h_ph)               
                self.axs[2].set_ylim([np.min(h_ph)-0.05*h_ph_range, np.max(h_ph)+0.05*h_ph_range] if h_ph_min != h_ph_max else [h_ph_min-1, h_ph_max+1])
                w_min = np.min(w)
                w_max = np.max(w)
                w_range = abs(w_max - w_min)
                self.axs[2].set_xlim([w_min-0.05*w_range, w_max+0.05*w_range] if w_min != w_max and w_min-0.05*w_range > 0 
                                     else [w_min, w_max+1])
            
            if self.active_line is not None:
                l_x, l_y = self.axs[0].lines[self.active_line + 1].get_data()
            if len(l_y) > self.active_point and len(l_x) > self.active_point:
                self.actual_change = False
                self.lastzeroRe = round(l_x[self.active_point], 3)
                self.input_Zero_RE.value = self.lastzeroRe
                self.lastzeroIm = round(l_y[self.active_point], 3)
                self.input_Zero_IM.value = self.lastzeroIm
                self.actual_change = True
    
    def unit_circle(self):
        # Generate a unit circle plot
        x1 = np.linspace(-1, 1, 1000)
        y1 = np.sqrt(1-x1**2)
        x2 = np.linspace(0.999, -1, 1000)
        y2 = np.sqrt(1-x2**2)
        x = np.concatenate([x1, x2])
        y = np.concatenate([y1, -y2])
        labels = ['-2j', '-1.5j', '-j', '-0.5j', '0', '0.5j', 'j','1.5j','2j']
        position=[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2]
        plt.yticks(position, labels, rotation='horizontal')
        return x, y

In [73]:
plt.close('all')
dzpp = DraggableZeroPolePlot()

In [ ]:
DraggableZeroPolePlot.change_freq_res.w

In [ ]:
z = np.array([])
p = np.array([0.5 - 0.5j])
gain = np.prod(1-z) / np.prod(1-p)
H = signal.ZerosPolesGain(z, p, gain).to_tf()

# Generate the continuous frequency response
try:
    w, h, h_ph = H.bode(n=1000)
except ValueError:
    print('Error')
plt.figure()
plt.semilogx(w, h)